# Text Parser Testing document

This document is going to read the text file found in ../generated_data/Logs2020OCR_avepdf.com_horizontal.txt which was generated using the following website:


[avepdf.com](http://avepdf.com/)

In [1]:
import re # Regulra expresion library

In [2]:
filename='../generated_data/Logs2020OCR_avepdf.com_horizontal.txt'
#filename='../generated_data/Logs2019OCR_avepdf.com_horizontal.txt'

In [3]:
# read file in as a list of lines
def readfile(filename):
    lines = []
    with open(filename) as f:
        lines = f.readlines()

    for ind in range(len(lines)):
        lines[ind] = lines[ind].lstrip().rstrip()
    return lines

lines = readfile(filename)

In [4]:
# Parse cells using 20-1 regular expresisons
# Result is a list of list where each call is a list of lines for the call.

def txt2calls(lines):
    calls = []

    call = []
    for line in lines:
        if line == '':
            pass
        else:
            if re.match("[1-2][9,0]-[0-9]+\s", line[:10]):
                if len(call) > 0:
                    calls.append(call)
                    call = []
            call.append(line)
    if len(call) > 0:
        call.append(line)
    return calls

calls = txt2calls(lines)

In [16]:
def parseHeader(line):
    '''Parse the first line of each call. Use error checking to find components.'''
    print(line)
    index_past = 0
        
    callNumberMatch = re.search("[19,20]-\d+", line)
    if callNumberMatch == None:
        return

    if callNumberMatch is not None:
        callNumber = callNumberMatch.group()
        index_past = callNumberMatch.span()[1] + 1
    else:
        callNumber = None
        
    timeMatch = re.search("\d\d\d\d", line[index_past:])
    if timeMatch is not None:
        callTime = timeMatch.group()
        index_past += timeMatch.span()[1]+1
    else:
        callTime = None
        
    spaces = 0
    index = len(line)-1
    while spaces < 8:
        if index > 0 and index < len(line):
            if line[index] == " ":
                spaces += 1
            else:
                spaces = 0
        else:
            break
        index -= 1
    callReason = re.sub("\s+", " ", line[index_past:index])
    if len(callReason) > 1:
        if callReason[0] == " ":
            callReason = callReason[1:]
        if callReason[-1] == " ":
            callReason = callReason[:-1]
    else:
        callReason=""
    
    callAction = re.sub("\s+", " ", line[index+8:])
    if len(callAction) > 1:
        if callAction[0] == " ":
            callAction = callAction[1:]
        if callAction[-1] == " ":
            callAction = callAction[:-1]
    else:
        callAction=""
        
    return [callNumber, callTime, callReason, callAction]

def get_unit_times(unit_str):
#     print(unit_str)
    times = re.sub(' +', ' ',unit_str).split(' ')
    tm_dict = {}
    for tm in times:
        if '-' in tm:
            vals = tm.split('-')
            tm_dict[vals[0]] = vals[-1]
    return tm_dict

def parse_call_list(call):
    '''Parse a call list into a dictionary'''
    
    if len(call) == 0:
        return
    
    my_call = {}
    
    header = parseHeader(call[0])
    if header: 
        my_call['callNumber'] = header[0]
        my_call['callTime'] = header[1]
        my_call['callReason'] = header[2]
        my_call['callAction]']= header[3]
#     my_call['header'] = call[0]
    
    ind = 0
    individual = ''
    while ind < len(call):
        line = call[ind]
        myline = line.split(':')
        if len(myline) == 2:
            tag = myline[0].rstrip().lstrip()
            tag = re.sub(' +', ' ',tag)
            value = myline[1].rstrip().lstrip()
            if tag == 'Narrative':
                if tag in my_call:
                    narrative = my_call['Narrative']
                else:
                    narrative = ''
                ind +=1
                while ind < len(call):
                    myline = call[ind].split(':')
                    if len(myline) == 1:
                        narrative += re.sub(' +', ' ',call[ind]) + " "
                    ind += 1
                my_call['Narrative'] = narrative
            else:
                if tag == "Unit":
                    ind += 1;
                    if ind < len(call):
                        unit_info = get_unit_times(call[ind])
                        if 'Units' in my_call:
                            my_call['Units'].append((value, unit_info))
                        else:
                            my_call['Units'] = [(value, unit_info)]
                else:
                    if tag == "Operator" or tag == "Owner":
                        individual=tag+"_"
                    my_call[tag] = value
        else:
            if len(myline) > 2:
                tags = re.findall('[\S]+:', line)

                for tag in reversed(tags):
                    start = line.rindex(tag)
                    value = line[start+len(tag):].rstrip().lstrip()

                    if len(value) > 0:
                        tag = tag[:-1]
                        my_call[individual+tag] = value
                        line = line[:line.rindex(tag)]
        ind +=1
    return(my_call)

In [17]:
def parse_all_calls(calls):
    '''Parse all of the calls'''
    call_dicts =[]
    unit = ''
    for call in calls:
        my_call = parse_call_list(call)
        call_dicts.append(my_call)
    return call_dicts

call_dicts = parse_all_calls(calls)

1
20-1                          0036         Initiated         - MOTOR  VEHICLE  STOP                              CITATION   - WARNING
20-2                          0128         Phone  - B.O.L.O.                                                                   SERVICES RENDERED
20-3                          0201         Phone   - B.O.L.O.                                                                  UNABLE TO LOCATE
20-4                         0204         911   - ALARM   - BURGLAR                                                  ALARM FALSE
20-6                         0253         911   - ASSIST    OTHER AGENCY-   FIRE                           SERVICES RENDERED
20-8                          0412         Initiated         - PARKING   CHECK                                        SERVICES RENDERED
20-10                        0744         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-11                       0820         

20-1633                    2206         911   - DISTURBANCE                                                           SERVICES RENDERED
20-1636                    0348         Initiated         - PARKING   CHECK                                        SERVICES RENDERED
20-1637                   0723         Initiated          - TRAFFIC    CONTROL                                   SERVICES RENDERED
20-1638                   0736         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-1639                        0746            Initiated            - TRAFFIC      CONTROL                                          SERVICES    RENDERED
20-1640                        0759           Initiated             - BUILDING       CHECK                                            BUILDING     CHECKED/SECURED
20-1641                        0802           Initiated            - BUILDING       CHECK                                             BUILDING 

20-2671                    0337         Initiated         - PARKING   CHECK                                        SERVICES RENDERED
20-2673                    0844         Initiated         - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-2674                    0847         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-2675                   0853          Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-2676                    0854         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-2677                    0857         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-2678                    0900         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-2679         

20-4113                   2059         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-4115                   0320          Initiated         - PARKING   CHECK                                        SERVICES RENDERED
20-4117                    0806         Phone  - ROAD   CONDITIONS                                             TAKEN/REFERRED TO OTHER
20-4118                    0817         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-4119                    0819         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-4120                   0838         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-4121                   0840          Initiated         - MOTOR VEHICLE  STOP                              CITATION   - CIVIL
20-4122                    0845       

20-5394                   0351          Initiated         - PARKING  CHECK                                         SERVICES RENDERED
20-5395                   0457          911  - ALARM   - BURGLAR                                                  ALARM FALSE
20-5396                   0632          911  - 9-1-1       TEST CALL                                                  NO ACTION  REQUIRED
20-5397                   0705         Phone   - ASSIST    OTHER AGENCY - UTILITY                  SERVICES  RENDERED
20-5399                   0809         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-5400                   0814         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-5401                   0831          Initiated         - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-5402                   0833         Initiated          - SP

20-6000                    0931         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-6001                   1048         Phone   - DISTURBANCE                                                       LOG ENTRY REQUEST
20-6003                   1448         Walk-In     - TRAFFIC    COMPLAINT                                    PATROL ADVISED
20-6005                   1517         911   - 9-1-1      WRONG NUMBER                                        SERVICES RENDERED
20-6010                   1624          911  - 9-1-1      WRONG NUMBER                                        TAKEN/REFERRED TO OTHER
20-6011                    1659         911  - 9-1-1      WRONG NUMBER                                        SERVICES RENDERED
20-6012                   1742          911  - 9-1-1      WRONG NUMBER                                        SERVICES RENDERED
20-6013                   1812         Initiated          - BUILDING    CHECK      

20-6876                   1916         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-6877                   2051         Initiated         - BUILDING     CHECK                                     BUILDING   CHECKED/SECURED
20-6878                   2341         Phone   - ALARM-     BURGLAR                                            ALARM FALSE
20-6882                    0334        Initiated          - PARKING   CHECK                                        SERVICES RENDERED
20-6883                    0656        Phone   - SUSPICIOUS   MOTOR VEHICLE                          GONE ON ARRIVAL
20-6885                    0838         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-6886                   0844         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-6887                    0848         Initiated         - BU

20-8069                   0933         Phone   - WELL   BEING  CHECK                                          SERVICES RENDERED
20-8071                    1241         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-8075                    1607         Initiated         - BUILDING    CHECK                                      SERVICES RENDERED
20-8077                   1735          911  - 9-1-1      WRONG NUMBER                                        SERVICES RENDERED
20-8079                   1836         Phone   - TRAFFIC    COMPLAINT                                        UNABLE TO  LOCATE
20-8080                    1901         Phone   - TRAFFIC   COMPLAINT                                         TAKEN/REFERRED TO OTHER
20-8081                    1959         Initiated         - BUILDING    CHECK                                      SERVICES RENDERED
20-8082                    1959         Initiated         - MOTOR VEHICLE  

In [18]:
def parsefile(filename):
    lines = readfile(filename)
    calls = txt2calls(lines)
    call_dicts = parse_all_calls(calls)
    return call_dicts

In [24]:
filename='../generated_data/Logs2019OCR_avepdf.com_horizontal.txt'
dict2019 = parsefile(filename)
filename='../generated_data/Logs2020OCR_avepdf.com_horizontal.txt'
dict2020 = parsefile(filename)

call_dicts = dict2019+dict2020

1
19-1                          0341         Initiated         - PARKING   CHECK                                        SERVICES  RENDERED
19-4                         0834          Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-5                         0842         Initiated          - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-6                         0846         Initiated          - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-7                         0847         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
19-8                         0859         Initiated          - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-9                         0903         Initiated          - BUILDING    CHECK                                      BUILDING   CHECKED/

19-2000                    1033         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-2001                    1035         Initiated         - PARKING   CHECK                                        SERVICES RENDERED
19-2002                    1036         Initiated         - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
19-2003                   1039         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
19-2005                   1042         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-2006                   1043          Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-2007                   1045          Initiated         - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
19-2011            

19-3279                   0923         Initiated         - BUILDING     CHECK                                     BUILDING   CHECKED/SECURED
19-3280                   0926         Initiated          - BUILDING    CHECK                                     BUILDING   CHECKED/SECURED
19-3281                   0932         Initiated          - BUILDING    CHECK                                     BUILDING   CHECKED/SECURED
19-3282                   0937         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-3283                   0939         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-3284                    0940         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-3285                   0946         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
19-3286     

19-4967                   1042         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
19-4968                   1047         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
19-4969                   1101         Phone   - ANIMAL   CONTROL                                              TRANSPORTED TO SHELTER
19-4972                    1214         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-4974                    1225         Phone  - ASSIST     OTHER AGENCY - POLICE                  SERVICES RENDERED
19-4975                   1242         Initiated         - BUILDING     CHECK                                     BUILDING   CHECKED/SECURED
19-4976                    1244         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-4977                   1247         Ini

19-6271                   0659         911   - 9-1-1      ABANDONED CALL                                     UNFOUNDED
19-6272                   0806         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-6273                    0811        Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
19-6277                    0845        Walk-In      - LARCENY                                                           INVESTIGATED/     REPORT
19-6276                   0858         Phone   - MOTOR VEHICLE    ACCIDENT                            INVESTIGATED/     REPORT
19-6278                   0926         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-6279                   0947         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
19-6280                   0952         Initia

...

20-2841                    1503         Phone   - LARCENY                                                               LOG ENTRY REQUEST
20-2840                    1504         Initiated         - MOTOR VEHICLE  STOP                              CITATION   - WARNING
20-2842                    1513         Initiated         - PARKING   CHECK                                        SERVICES RENDERED
20-2843                   1520          Initiated         - PARKING  CHECK                                         SERVICES RENDERED
20-2845                   1539          Initiated         - MOTOR VEHICLE  STOP                              CITATION   - WARNING
20-2847                   1611         Initiated          - BUILDING    CHECK                                      SERVICES RENDERED
20-2849                   1712         Initiated          - PARKING  CHECK                                         SERVICES RENDERED
20-2850                   1716         Walk-In      - LICENSE   TO  CA

20-4384                   2204          Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-4385                   2226         Phone   - ALARM - BURGLAR                                                ALARM FALSE
20-4386                    0337         Initiated         - PARKING   CHECK                                        SERVICES  RENDERED
20-4388                    0745         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-4389                    0747         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-4390                   0753         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-4391                    0757         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-4392                    0800  

20-5978                   0831         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-5981                    0958         Initiated         - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-5983                    1212        Other     - PUBLIC   RECORDS REQUEST                            SERVICES RENDERED
20-5986                    1355        Phone   - MOTOR VEHICLE  LOCK OUT                              VEHICLE  OPENED
20-5988                    1627        Walk-In      - MOTOR VEHICLE    ACCIDENT                        INVESTIGATED/     REPORT
20-5989                    1800         Initiated         - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-5990                    1923        Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-5991                   1936         Initiated          - BUILDING 

20-7293                    1634         Phone  - TRAFFIC    COMPLAINT                                         INVESTIGATED/     REPORT
20-7295                    1843         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-7296                    1850        Phone   - TRAFFIC    COMPLAINT                                         SERVICES RENDERED
20-7298                   2152         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-7299                   2157         Initiated          - BUILDING    CHECK                                      BUILDING  CHECKED/SECURED
20-7300                   0056         Other    - ASSIST    OTHER AGENCY - POLICE                   PATROL ADVISED
20-7301                    0253         Initiated         - PARKING  CHECK                                         SERVICES RENDERED
20-7302                    0729         Phone  - TRAFFIC    COMPL

20-9026                   0815         Initiated          - TRAFFIC   CONTROL                                    SERVICES RENDERED
20-9027                   0816         Initiated          - TRAFFIC   CONTROL                                    SERVICES RENDERED
20-9028                   0851         Initiated          - BUILDING    CHECK                                      SERVICES RENDERED
20-9029                   0908         Other    - PUBLIC    RECORDS REQUEST                            SERVED IN  HAND
20-9030                   0909         Other    - PUBLIC    RECORDS REQUEST                            SERVICES RENDERED
20-9032                   0915         Other    - PUBLIC    RECORDS REQUEST                            SERVICES RENDERED
20-9033                   1006         Initiated         - BUILDING    CHECK                                      BUILDING   CHECKED/SECURED
20-9034                    1015         Initiated         - BUILDING    CHECK                          

In [25]:
#Count number of STOPS by "Craig"
total_stops = 0 
tot = 0
total_race = {}
norace = 0
my_stops = []
for call in call_dicts:
    if 'Call Taker' in call:
        if "hammer" in call['Call Taker'].lower():
            tot += 1
#             print(call['callReason'])
            if "STOP" in call['callReason']:
                my_stops.append(call)
                total_stops += 1
                if 'Operator_Race' in call:
                    race = call['Operator_Race'].lower()
                    if race in total_race:
                        total_race[race] = total_race[race]+1
                    else:
                        total_race[race] = 1
                else:
                    norace +=1

In [26]:
total_stops

23

In [27]:
tot

394

In [28]:
total_race

{'u': 14, 'w': 7}

In [29]:
#Count number of STOPS by "Craig"
alltags = {}

for call in call_dicts:
    for tag in call:
        if tag in alltags:
            alltags[tag] = alltags[tag] + 1
        else:
            alltags[tag] = 1

In [30]:
alltags

{'For Date': 175,
 'callNumber': 18978,
 'callTime': 18978,
 'callReason': 18978,
 'callAction]': 18978,
 'Call Taker': 18368,
 'Location/Address': 17814,
 'Units': 16300,
 'Narrative': 13318,
 'VIN': 3622,
 'Reg': 3827,
 'Vehicle': 4126,
 'Operator': 3425,
 'Operator_Sex': 3201,
 'Operator_Race': 3282,
 'Refer To Citation': 2750,
 'Owner': 3742,
 'Owner_Sex': 3253,
 'Owner_Race': 3329,
 'Owner_sex': 82,
 'Owner_VIN': 215,
 'Owner_Reg': 222,
 'Owner_Vehicle': 220,
 'Refer ·To Citation': 1,
 'Location': 109,
 '[Modified': 25,
 'Change': 25,
 'Clrd-09:33': 2,
 'Arvd-09:27': 1,
 'owner': 381,
 'Clrd-17:30': 1,
 'Arvd-17:29': 1,
 'Refer To Incident': 21,
 'Uhit': 1,
 'Clrd-18:46': 1,
 'Arvd-18:44': 1,
 'To': 103,
 'By': 108,
 'For': 111,
 "Clrd-11'56": 1,
 '54': 1,
 'Enrt-11:51': 1,
 'Disp-11:51': 1,
 'Clrd-22:42': 1,
 'Arvd-22:40': 1,
 'Enrt-22:35': 1,
 'Disp-22:33': 1,
 '00:01': 6,
 'Clrd-03:38': 1,
 'Arvd-03:38': 1,
 'Operator_Reg': 36,
 'Operator_Vehicle': 34,
 'Clrd-10:57': 2,
 'Arvd-